# Scoring images on Spark

This notebook illustrates how trained Cognitive Toolkit (CNTK) and TensorFlow models can be applied to large image collections using PySpark. For more detail on image set creation and model training, please see the rest of the [Embarrassingly Parallel Image Classification](https://github.com/Azure/Embarrassingly-Parallel-Image-Classification) repository.

## Outline
- [Setting up a Microsoft HDInsight Spark cluster and Azure Data Lake Store](#setup)
   - [Provisioning the resources](#provision)
   - [Transferring the image set](#transfer)
   - [Installing Cognitive Toolkit and Tensorflow](#install)
- [Image scoring with PySpark](#pyspark)
   - [Cognitive Toolkit](#cntk)
   - [TensorFlow](#tf)

<a name="setup"></a>
## Setting up a Microsoft HDInsight Spark cluster and associated Azure Data Lake Store

<a name="provision"></a>
### Provisioning the resources

#### Azure Data Lake Store

#### Azure HDInsight Spark Cluster
1. After logging into [Azure Portal](https://ms.portal.azure.com), click the "+ New" button near the upper left to create a new resource.
   <img src="./img/spark_adls_provisioning/new_resource_button.gif" width="100 px"/>
1. In the search field that appears, enter "HDInsight" and press Enter.
   <img src="./img/spark_adls_provisioning/resource_search_box.gif" width="200 px"/>

1. In the search results, click on the "HDInsight" option published by Microsoft.
   <img src="./img/spark_adls_provisioning/resource_search_result.gif" width="400 px"/>

1. Click the "Create" button at the bottom of the new pane that opens to describe the HDInsight resource type.
   <img src="./img/spark_adls_provisioning/create.gif" width="100 px"/>
1. In the "Basics" section of the "New HDInsight cluster" pane:
    1. Choose a unique cluster name and the appropriate subscription.
    1. Click on "Cluster configuration" to load a pane of settings.
       1. Set the cluster type to "Spark".
       1. Set the version to "Spark 2.0.2 (HDI 3.5)".
       <img src="./img/spark_adls_provisioning/cluster_type_settings.gif" width="400 px"/>
       1. Click the "Select" button at the bottom of the pane.
    1. Choose a password for the `admin` account. You will use this account to log into Jupyter Notebook later in the walkthrough.
    1. Select or create an appropriate resource group.
    1. Click the "Next" button at the bottom of the pane.
1. In the "Storage" section of the "New HDInsight cluster" pane:
   1. Ensure that "Data Lake Store" is selected for the "Primary storage type".
   1. Click on "Select Data Lake Storage Account" to load a pane of settings.
       1. Under "Select a storage account", select your ADLS.
       1. Click on "Configure Data Lake Access" to load a pane of settings.
           1. Create a new service principal.
           1. Click on "Access" to load a pane of settings.
               1. Under "Select File Permissions", click the box to the left of your ADLS name. (The box may be obscured until mouseover.). Click "Select".
               1. Under "Assign Selected Permissions", click "Run".
               1. When the run completes, click "Done".
       1. Click the "Next" button at the bottom of the pane.
1. In the "Summary" section of the "New HDInsight cluster" pane:
   1. If desired, you can edit the cluster size settings to choose node counts/sizes based on your budget and time constraints. This tutorial can be completed using a cluster with **4** worker nodes and a node size of **D12 v2** (for both worker and head nodes). For more information, please see the [cluster](https://docs.microsoft.com/en-us/azure/hdinsight/hdinsight-hadoop-provision-linux-clusters) and [VM](https://docs.microsoft.com/en-us/azure/virtual-machines/virtual-machines-linux-sizes#dv2-series) size guides.
1. Click the "Create" button at the bottom of the pane.

Cluster deployment may take approximately twenty minutes. (Since Azure Data Lake Store deployment will finish much sooner, we recommend transferring your image set to the ADLS while you wait; see the next section.) Cluster deployment status can be checked as follows:
1. Click on the "Search Resources" magnifying glass icon along the top bar of [Azure Portal](https://ms.portal.azure.com).
1. Type in the name of your HDInsight cluster and click on its entry in the resulting drop-down list. The overview pane for your HDInsight cluster will appear.
1. During deployment, a blue bar will appear across the top of the overview pane with the title "Applying changes". When this bar disappears, deployment is complete.

<a name="transfer"></a>
### Transferring the image set

Our evaluation image set was creating on a Data Science Virtual Machine. To transfer these images to our Azure Data Lake Store, we first copied the images to Azure Blob Storage using [AzCopy](https://docs.microsoft.com/en-gb/azure/storage/storage-use-azcopy), then to the Azure Data Lake Store with [AdlCopy](https://docs.microsoft.com/en-us/azure/data-lake-store/data-lake-store-copy-data-azure-storage-blob). After following the instructions linked above to download and install AzCopy/AdlCopy, we transferred the files with the following shell commands:

In [3]:
local_image_dir = 'E:\\combined\\test'
blob_account_name = 'mawahstorage'
blob_account_key = 'o62OKYWfsL/sNki1udZPWUZkOY5y6tL7cLRlgDTMciO9ZavfwmKqa8vNTNwrJXqkjeqHl9wJULwowfQFkj4/JA=='
blob_account_container = 'testing'
adl_account_name = 'mawahtensorflow'
adl_account_folder = 'testing'

commands = '''
AzCopy /Source:{0} /Dest:https://{1}.blob.core.windows.net/{2} /DestKey:{3} /S
AdlCopy /source https://{1}.blob.core.windows.net/{2}/ /dest swebhdfs://{4}.azuredatalakestore.net/{5}/ /sourcekey {3}
'''.format(local_image_dir, blob_account_name, blob_account_container,
           blob_account_key, adl_account_name, adl_account_folder)

print(commands)


AzCopy /Source:E:\combined\test /Dest:https://mawahstorage.blob.core.windows.net/testing /DestKey:o62OKYWfsL/sNki1udZPWUZkOY5y6tL7cLRlgDTMciO9ZavfwmKqa8vNTNwrJXqkjeqHl9wJULwowfQFkj4/JA== /S
AdlCopy /source https://mawahstorage.blob.core.windows.net/testing/ /dest swebhdfs://mawahtensorflow.azuredatalakestore.net/testing/ /sourcekey o62OKYWfsL/sNki1udZPWUZkOY5y6tL7cLRlgDTMciO9ZavfwmKqa8vNTNwrJXqkjeqHl9wJULwowfQFkj4/JA==



<a name="install"></a>
### Installing Cognitive Toolkit and Tensorflow

#### Obtaining and (optionally) modifying the script action

We will install Cognitive Toolkit and Tensorflow on all head and worker nodes via Script Action. We have included a sample script action in the `scoring` subdirectory of [the Embarrassingly Parallel Image Classification git repository](https://github.com/Azure/Embarrassingly-Parallel-Image-Classification), reproduced below for your convenience:

The code above installed CNTK 2.0 beta release 10. As of this writing, other CNTK releases can be substituted as follows:
1. Navigate to the [CNTK Releases](https://github.com/Microsoft/CNTK/releases) page
1. Click on the appropriate release's link for a Linux, CPU Only release.
1. After reading and agreeing to the mentioned licenses, copy the URL linked to the "I accept" button (e.g. from the page source) and paste over the URL in the `curl` command above.

#### Running the script action

After HDInsight cluster deployment finishes, run the script action to install CNTK as follows:
1. Obtain the URI for the script action.
   - If using the unmodified version in this git repo, ensure that your URI points to the "raw" file (not a webpage-embedded file).
   - If you have modified the script action, upload it to the website or Azure Blob Storage account of your choice and note its URI.
1. Click on the "Search Resources" magnifying glass icon along the top bar of [Azure Portal](https://ms.portal.azure.com).
1. Type in the name of your HDInsight cluster and click on its entry in the resulting drop-down list. The overview pane for your HDInsight cluster will appear.
1. In the search field at upper left, type in "Script actions". Click the "Script actions" option in the results list.
1. Click the "+ Submit new" button along the top of the Script Actions pane. A new pane of options will appear.
   1. Under name, type "install" (without the quotes).
   1. Under "Bash script URI", type in the URI.
   1. Ensure that "Head" and "Worker" boxes are checked.
   1. Click the "Create" button along the bottom of the pane.
   
Expect the script action to take roughly fifteen minutes to run.
   
#### Updating the Python 3 path

The script action above installed Cognitive Toolkit and Tensorflow under a new Python environment, `cntk-py35`. Follow the steps below to direct PySpark to use this new environment:

1. Navigate back to the HDInsight cluster's overview pane by clicking "Overview" near the upper left of the pane.
1. Under "Quick links" in the main window, click the "Cluster dashboards" button. A new pane of dashboard options will appear.
1. Click "HDInsight cluster dashboard". A new window will load.
1. In the menu at left, click "Spark2".
1. In the main window, click on the "Configs" tab.
1. Scroll down to the "Custom spark2-defaults" option and expand its dropdown by clicking on the label (or triange beside it).
1. Find the `spark.yarn.appMasterEnv.PYSPARK3_PYTHON` entry in the dropdown list. Change its path to the following:

    `/usr/bin/anaconda/envs/cntk-py35/bin/python`<br/><br/>
    
1. Click on the green "Save" button that appears at upper right.
1. When prompted, click the orange "Restart" button and select "Restart all affected".
1. When the restart concludes, close the window. This will return you to a pane of dashboard options.

<a name="pyspark"></a>
## Image scoring with PySpark

### Define common variables, including image RDD

The following variables will be referenced by 

In [4]:
adls_name = 'mawahtensorflow'
adls_folder = 'testing'
n_workers = 4
local_tmp_dir = '/tmp/resnet'

dataset_dir = 'adl://{}.azuredatalakestore.net/{}'.format(adls_name, adls_folder)
image_rdd = sc.binaryFiles('{}/*/*.png'.format(dataset_dir), minPartitions=n_workers).coalesce(n_workers)

<a name="cntk"></a>
### Cognitive Toolkit (CNTK)

#### Download the trained CNTK model

Use the code segment below to download the trained CNTK ResNet and ensure the file will be accessible to workers:

In [ ]:
cntk_model_uri = 'https://mawahstorage.blob.core.windows.net/examples/cntkonspark/resnet20_159.dnn'

with open('{}/model.dnn'.format(local_tmp_dir), 'wb') as f:
    f.write(urlopen(cntk_model_uri).read())
sc.addFile('{}/model.dnn'.format(local_tmp_dir))

#### Create an RDD of image data



<a name="tf"></a>
### Tensorflow

### Get the trained Tensorflow model

TODO: link to repo

In [ ]:
tf_model_uri = 'https://mawahstorage.blob.core.windows.net/examples/cntkonspark/resnet20_159.dnn'

with open('{}/model.dnn'.format(local_tmp_dir), 'wb') as f:
    f.write(urlopen(cntk_model_uri).read())
sc.addFile('{}/model.dnn'.format(local_tmp_dir))

### Perform imports:

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import functools
from io import BytesIO
from PIL import Image

sc.addPyFile('/tmp/resnet/tf/resnet_utils.py')
sc.addPyFile('/tmp/resnet/tf/resnet_v1.py')
import resnet_v1  # which has been modified based on an identified git issue

slim = tf.contrib.slim

In [ ]:
##############################
# Head node helper functions #
##############################

def read_label_file(dataset_dir, filename='labels.txt'):
    labels_file = sc.wholeTextFiles('{}/{}'.format(dataset_dir, filename)).take(1)[0][1]
    lines = labels_file.split('\n')
    lines = filter(None, lines)
    labels_to_class_names = {}
    for line in lines:
        index = line.index(':')
        labels_to_class_names[int(line[:index])] = line[index+1:]
    return(labels_to_class_names)

In [ ]:
################################
# Worker node helper functions #
################################

def get_network_fn(name, num_classes, weight_decay=0.0, is_training=False):
    arg_scope = resnet_v1.resnet_arg_scope(weight_decay=weight_decay)
    func = resnet_v1.resnet_v1_152
    @functools.wraps(func)
    def network_fn(images):
        with slim.arg_scope(arg_scope):
            return func(images, num_classes, is_training=is_training)
    if hasattr(func, 'default_image_size'):
        network_fn.default_image_size = func.default_image_size
    return(network_fn)

def mean_image_subtraction(image, means):
    num_channels = image.get_shape().as_list()[-1]
    channels = tf.split(2, num_channels, image)
    for i in range(num_channels):
        channels[i] -= means[i]
    return(tf.concat(2, channels))

def get_preprocessing():
    def preprocessing_fn(image, output_height=224, output_width=224):
        image = tf.expand_dims(image, 0)
        resized_image = tf.image.resize_bilinear(image, [output_height, output_width], align_corners=False)
        resized_image = tf.squeeze(resized_image)
        resized_image.set_shape([output_height, output_width, 3])
        image = tf.to_float(resized_image)
        return(mean_image_subtraction(image, [123.68, 116.78, 103.94]))
    return(preprocessing_fn)

def batch_label(files):    
    labels_to_class_names = labels_to_class_names_bc.value
    model_dir = model_dir_bc.value
    
    class_names = sorted(labels_to_class_names.values())
    class_names_to_ids = dict(zip(class_names, list(range(len(class_names)))))
    class_count = len(class_names)
    results = []
    
    with tf.Graph().as_default():
        network_fn = get_network_fn('resnet_v1_152', num_classes=class_count, is_training=False)
        image_preprocessing_fn = get_preprocessing()

        current_image = tf.placeholder(tf.uint8, shape=(100, 100, 3))
        preprocessed_image = image_preprocessing_fn(current_image, 224, 224)
        image  = tf.expand_dims(preprocessed_image, 0)
        logits, _ = network_fn(image)
        predictions = tf.argmax(logits, 1)

        with tf.Session() as sess:
            my_saver = tf.train.Saver()
            my_saver.restore(sess, tf.train.latest_checkpoint(model_dir))

            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            try:
                for file in files:
                    imported_image_np = np.asarray(Image.open(BytesIO(file[1])), dtype=np.uint8)
                    result = sess.run(predictions, feed_dict={current_image: imported_image_np})
                    class_name = file[0].split('/')[-2] # Extract class name from filename
                    results.append((result[0], class_names_to_ids[class_name]))
            finally:
                coord.request_stop()
            coord.join(threads)
    return(results)

In [ ]:
# Get the full list of images and label/class info
labels_to_class_names = read_label_file(dataset_dir)
class_names = sorted(labels_to_class_names.values())
class_names_to_ids = dict(zip(class_names, list(range(len(class_names)))))

# Broadcast some information to workers
local_tmp_dir_bc = sc.broadcast(local_tmp_dir)
labels_to_class_names_bc = sc.broadcast(labels_to_class_names)

# Split up the image list into n_workers partitions
labelled_images = image_rdd.mapPartitions(batch_label)
results = labelled_images.collect()